In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import gradient_descent_v2
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(r'C:\Users\Somesh\Desktop\AI_ChatBot_Python-master\AI_ChatBot_Python-master\intents.json').read()
intents = json.loads(data_file)

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
49 unique lemmatized words ["'d", "'s", 'a', 'accept', 'anyone', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodbye', 'have', 'hello', 'helpful', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'later', 'like', 'mastercard', 'moped', 'of', 'only', 'open', 'rent', 'see', 'take', 'thank', 'thanks', 'that', 'there', 'this', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'you', 'your']


In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\Somesh\AppData\Local\Temp/ipykernel_7220/2836083368.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = gradient_descent_v2.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

C:\Users\Somesh\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 3s 10ms/step - loss: 2.1882 - accuracy: 0.1111
Epoch 2/200
6/6 [==============================] - 0s 9ms/step - loss: 2.1672 - accuracy: 0.1852
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 2.1806 - accuracy: 0.1481
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0898 - accuracy: 0.3333
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.9260 - accuracy: 0.4074
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8960 - accuracy: 0.3333
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.9872 - accuracy: 0.4074
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 1.8547 - accuracy: 0.2593
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 1.7354 - accuracy: 0.4815
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7069 - accuracy: 0.5926
Epoch 11/200
6/6 [==========

6/6 [==============================] - 0s 4ms/step - loss: 0.1036 - accuracy: 1.0000
Epoch 85/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0859 - accuracy: 0.9630
Epoch 86/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1799 - accuracy: 0.9630
Epoch 87/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0524 - accuracy: 1.0000
Epoch 88/200
6/6 [==============================] - 0s 4ms/step - loss: 0.1144 - accuracy: 1.0000
Epoch 89/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1562 - accuracy: 1.0000
Epoch 90/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1921 - accuracy: 0.9630
Epoch 91/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1104 - accuracy: 0.9630
Epoch 92/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1145 - accuracy: 1.0000
Epoch 93/200
6/6 [==============================] - 0s 2ms/step - loss: 0.1036 - accuracy: 0.9630
Epoch 94/200
6/6 [===============

6/6 [==============================] - 0s 4ms/step - loss: 0.0492 - accuracy: 0.9630
Epoch 168/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0277 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0425 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0450 - accuracy: 0.9630
Epoch 171/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0172 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 173/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0180 - accuracy: 1.0000
Epoch 174/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0151 - accuracy: 1.0000
Epoch 175/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0520 - accuracy: 1.0000
Epoch 176/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 177/200
6/6 [=====